# Neural Network from Scratch
The following neural network code was adapted from http://adventuresinmachinelearning.com/neural-networks-tutorial/; however, the website is no longer accessible.

The notation that was used in this website is almost the same as the notation we are using in class.  Instead of using $a$ for activations the author uses $h$, and instead of $N$ for the number of training examples, the author uses $m$. (I have modified the code below to use $a$ and $N$.)


#  Your assignment:

Modify the neural network implementation below (it is the same implementation that we discussed in class) to see if you can improve the performance by trying the following:


* Add a regularization term to the cost function. 
$$ J(W,b)
= \frac{1}{N}
{\Big [}\sum_{i=1}^N
 J(W,b,{\bf x}^{(i)},y^{(i)}){\Big ]}
+ \frac{\lambda}{2}\sum_{\ell}\sum_{i}\sum_j (W^{(\ell)}_{ij})^2$$

The partial derivative would be: 
$$\frac{\partial J(W,b)}{\partial W^{(\ell)}_{ij}}
= \frac{1}{N}
{\Big [}\sum_{i=1}^N
\frac{\partial J(W,b,{\bf x}^{(i)},y^{(i)})}{\partial W^{(\ell)}_{ij}} {\Big ]}
+ {\lambda} W^{(\ell)}_{ij}$$ where ${\bf x}^{(i)},y^{(i)}$ are the $i$th training example.  <br><br>

* Try using the **ReLU** activation function, $f(z) = \max(0,z)$. You will notice it is not differentiable at $0$, but you can use: $f'(z) = 0$ if $z<0$ and $f'(z)=1$ if $z\ge 0$.  (You can also try using the **leaky ReLU** activation function.)  For more information see https://www.kaggle.com/dansbecker/rectified-linear-units-relu-in-deep-learning.  (The performance of the ReLU may surprise you.) <br><br>

* Try using the **tanh** activation function, $f(z) = \frac{e^z-e^{-z}}{e^z+e^{-z}}$.  The derivative of **tanh** is $f'(z)=1-(f(z))^2$.  For more information see http://ufldl.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks/ <br><br>
* Try using the one of the other activation functions **ELU** (exponential linear unit) $ELU_{\alpha}(z)= 
\begin{cases}  \alpha(\exp (z)-1) & \text{if } z<  0 \\ z & \text{if } z\ge 0 \end{cases}$
* Try changing the number of iterations
* Try changing the number of hidden layers
* Try initializing the weights using one of the techniques mentioned in the lecture


For each of the modifications about, describe your experience (i.e. what did you try and how well did it peform).  What gave the best performance?


## The first thing we will do is import all the libraries

We will be using the lower resolution MINST data set

In [ ]:
from sklearn.datasets import load_digits # The MNIST data set is in scikit learn data set
from sklearn.preprocessing import StandardScaler  # It is important in neural networks to scale the date
from sklearn.model_selection import train_test_split  # The standard - train/test to prevent overfitting and choose hyperparameters
from sklearn.metrics import accuracy_score # 
import numpy as np
import numpy.random as r # We will randomly initialize our weights
import matplotlib.pyplot as plt 

## Looking at the data

After we load the data, we print the shape of the data and a pixelated digit.

We also show what the features of one example looks like.

The neural net will learn to estimate which digit these pixels represent.

In [ ]:
digits=load_digits()
X = digits.data
print("The shape of the digits dataset:") 
print(digits.data.shape)
plt.gray()
plt.matshow(digits.images[0])
plt.show()
y = digits.target
print(y[0:1])
print(X[0,:])

## 1) Scale the dataset
The training features range from 0 to 15.  To help the algorithm converge, we will scale the data to have a mean of 0 and unit variance

In [ ]:
X_scale = StandardScaler()
X = X_scale.fit_transform(digits.data)

X[0,:] # Looking the new features after scaling

## 2) Creating training and test datasets
We split the data into training and test data sets. We will train the neural network with the training dataset, and evaluate our neural network with the test dataset 

In [ ]:
#Split the data into training and test set.  60% training and %40 test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

## 3) Setting up the output layer

### One-vs-all encoding
Our target is an integer in the range [0,..,9], so we will have 10 output neuron's in our network.  

-  If  $y=0$, we want the output neurons to have the values $(1,0,0,0,0,0,0,0,0,0)$

-  If  $y=1$ we want the output neurons to have the values $(0,1,0,0,0,0,0,0,0,0)$
-  etc

Thus we need to change our target so it is the same as our hoped for output of the neural network.  
-  If $y=0$ we change it into the vector $(1,0,0,0,0,0,0,0,0,0)$. 
-  If $y=1$ we change it into the vector $(0,1,0,0,0,0,0,0,0,0)$
-  etc

See page 29 from the website listed above

The code to covert the target vector.

In [ ]:
def convert_y_to_vect(y):
    y_vect = np.zeros((len(y), 10))
    for i in range(len(y)):
        y_vect[i, y[i]] = 1
    return y_vect

Converting the training and test targets to vectors 

In [ ]:
# convert digits to vectors
y_v_train = convert_y_to_vect(y_train)
y_v_test = convert_y_to_vect(y_test)

A quick check to see that our code performs as we expect 

In [ ]:
print(y_train[0:4])
print(y_v_train[0:4])

## 4) Creating the neural network

### The activation function and its derivative

We will use the sigmoid activation function:  $f(z)=\frac{1}{1+e^{-z}}$

The deriviative of the sigmoid function is: $f'(z) = f(z)(1-f(z))$ 

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_deriv(z):
    return sigmoid(z) * (1 - sigmoid(z))

def relu(z):
    return np.maximum(0, z)

def relu_deriv(z):
    return np.where(z > 0, 1, 0)

alpha = 0.01

def leaky_relu(z):
    return np.where(z > 0, z, z * alpha)

def leaky_relu_deriv(z):
    return np.where(z > 0, 1, alpha)

def tanh(z):
    return np.tanh(z)

def tanh_deriv(z):
    return 1.0 - np.tanh(z)**2

def elu(z):
    return np.where(z >= 0, z, alpha * (np.exp(z) - 1))

def elu_deriv(z):
    return np.where(z >= 0, 1, alpha * np.exp(z))



f = tanh
f_deriv = tanh_deriv

### Creating and initialing W and b
We want the weights in W to be different so that during back propagation the nodes on a level will have different gradients and thus have different update values.

We want the  weights to be small values, since the sigmoid is almost "flat" for large inputs.

Next is the code that assigns each weight a number uniformly drawn from $[0.0, 1.0)$.  The code assumes that the number of neurons in each level is in the python list *nn_structure*.

In the code, the weights, $W^{(\ell)}$ and $b^{(\ell)}$ are held in a python dictionary

In [ ]:
def setup_and_init_weights_default(nn_structure):
    W = {} #creating a dictionary i.e. a set of key: value pairs
    b = {}
    for l in range(1, len(nn_structure)):
        W[l] = r.random_sample((nn_structure[l], nn_structure[l-1])) #Return “continuous uniform” random floats in the half-open interval [0.0, 1.0). 
        b[l] = r.random_sample((nn_structure[l],))
    return W, b

def setup_and_init_weights_uniform(nn_structure):
    W = {}
    b = {}
    bound = np.sqrt(6 / (nn_structure[0] + nn_structure[-1])) * 4
    for l in range(1, len(nn_structure)):
        W[l] = (r.random_sample((nn_structure[l], nn_structure[l-1])) * bound * 2) - bound
        b[l] = (r.random_sample((nn_structure[l],)) * bound * 2) - bound
    return W, b

def setup_and_init_weights_normal(nn_structure):
    W = {}
    b = {}
    sigma = np.sqrt(6 / (nn_structure[0] + nn_structure[-1])) * 4
    for l in range(1, len(nn_structure)):
        W[l] = r.normal(0, sigma, (nn_structure[l], nn_structure[l-1]))
        b[l] = r.normal(0, sigma, (nn_structure[l],))
    return W, b


setup_and_init_weights = setup_and_init_weights_default

### Initializing $\triangledown W$ and $\triangledown b$
Creating $\triangledown W^{(\ell)}$ and $\triangledown b^{(\ell)}$ to have the same size as $W^{(\ell)}$ and $b^{(\ell)}$, and setting $\triangledown W^{(\ell)}$, and  $\triangledown b^{(\ell)}$ to zero

In [ ]:
def init_tri_values(nn_structure):
    tri_W = {}
    tri_b = {}
    for l in range(1, len(nn_structure)):
        tri_W[l] = np.zeros((nn_structure[l], nn_structure[l-1]))
        tri_b[l] = np.zeros((nn_structure[l],))
    return tri_W, tri_b

## Feed forward
Perform a forward pass throught the network.  The function returns the values of $a$ and $z$

In [ ]:
def feed_forward(x, W, b):
    a = {1: x} # create a dictionary for holding the a values for all levels
    z = { } # create a dictionary for holding the z values for all the layers
    for l in range(1, len(W) + 1): # for each layer
        node_in = a[l]
        z[l+1] = W[l].dot(node_in) + b[l]  # z^(l+1) = W^(l)*a^(l) + b^(l)
        a[l+1] = f(z[l+1]) # a^(l+1) = f(z^(l+1))
    return a, z

## Compute $\delta$
The code below compute $\delta^{(s_l)}$ in a function called "calculate_out_layer_delta",  and  computes $\delta^{(\ell)}$ for the hidden layers in the function called "calculate_hidden_delta".  

If we wanted to have a different cost function, we would change the "calculate_out_layer_delta" function.


In [ ]:
def calculate_out_layer_delta(y, a_out, z_out):
    # delta^(nl) = -(y_i - a_i^(nl)) * f'(z_i^(nl))
    return -(y-a_out) * f_deriv(z_out) 


def calculate_hidden_delta(delta_plus_1, w_l, z_l):
    # delta^(l) = (transpose(W^(l)) * delta^(l+1)) * f'(z^(l))
    return np.dot(np.transpose(w_l), delta_plus_1) * f_deriv(z_l)

## The Back Propagation Algorithm


In [ ]:
def train_nn(nn_structure, X, y, iter_num=3000, alpha=0.25, lam=0.0):
    W, b = setup_and_init_weights(nn_structure)
    cnt = 0
    N = len(y)
    avg_cost_func = []
    print('Starting gradient descent for {} iterations'.format(iter_num))
    while cnt < iter_num:
        if cnt%1000 == 0:
            print('Iteration {} of {}'.format(cnt, iter_num))
        tri_W, tri_b = init_tri_values(nn_structure)
        avg_cost = 0
        for i in range(N):
            delta = {}
            # perform the feed forward pass and return the stored a and z values, to be used in the
            # gradient descent step
            a, z = feed_forward(X[i, :], W, b)
            # loop from nl-1 to 1 backpropagating the errors
            for l in range(len(nn_structure), 0, -1):
                if l == len(nn_structure):
                    delta[l] = calculate_out_layer_delta(y[i,:], a[l], z[l])
                    avg_cost += np.linalg.norm((y[i,:]-a[l]))
                else:
                    if l > 1:
                        delta[l] = calculate_hidden_delta(delta[l+1], W[l], z[l])
                    # triW^(l) = triW^(l) + delta^(l+1) * transpose(a^(l))
                    tri_W[l] += np.dot(delta[l+1][:,np.newaxis], np.transpose(a[l][:,np.newaxis])) # np.newaxis increase the number of dimensions
                    # trib^(l) = trib^(l) + delta^(l+1)
                    tri_b[l] += delta[l+1]
        # perform the gradient descent step for the weights in each layer
        for l in range(len(nn_structure) - 1, 0, -1):
            W[l] += -alpha * (1.0/N * tri_W[l] + lam * W[l])
            b[l] += -alpha * (1.0/N * tri_b[l])
        # complete the average cost calculation
        avg_cost = 1.0/N * avg_cost + lam/2.0 * sum([np.sum(W[l]**2) for l in W])
        avg_cost_func.append(avg_cost)
        cnt += 1
    return W, b, avg_cost_func


def predict_y(W, b, X, n_layers):
    N = X.shape[0]
    y = np.zeros((N,))
    for i in range(N):
        a, z = feed_forward(X[i, :], W, b)
        y[i] = np.argmax(a[n_layers])
    return y

## Running the neural network

Our code assumes the size of each layer in our network is held in a list.  The input layer will have 64 neurons (one for each pixel in our 8 by 8 pixelated digit).  Our hidden layer has 30 neurons (you can change this value).  The output layer has 10 neurons.

Next we create the python list to hold the number of neurons for each level and then run the neural network code with our training data.

This code will take some time...

In [ ]:
nn_structure = [64, 20, 10]
    
# train the NN
W, b, avg_cost_func = train_nn(nn_structure, X_train, y_v_train, iter_num=30000, alpha=0.25, lam=0.01)
print('Done')

### Plotting the learning curve


In [ ]:
# plot the avg_cost_func
plt.plot(avg_cost_func)
plt.ylabel('Average J')
plt.xlabel('Iteration number')
plt.show()

## 5) Assessing accuracy
Next we determine what percentage the neural network correctly predicted the handwritten digit correctly on the test set

In [ ]:
# get the prediction accuracy and print
y_pred = predict_y(W, b, X_test, 3)
print('Prediction accuracy is {}%'.format(accuracy_score(y_test, y_pred) * 100))

In [ ]:
'''
lams = [
    0,
    0.001,
    0.01,
    0.1,
    1.0,
]
for lam in lams:
    W, b, avg_cost_func = train_nn(nn_structure, X_train, y_v_train, iter_num=3000, alpha=0.25, lam=lam)
    plt.plot(avg_cost_func)
    plt.ylabel('Average J')
    plt.xlabel('Iteration number')
    plt.title('lambda = {}'.format(lam))
    plt.show()
    y_pred = predict_y(W, b, X_test, 3)
    print('Prediction accuracy for lambda = {} is {}%'.format(lam, accuracy_score(y_test, y_pred) * 100))
'''  

## My notes and conclusions

#### 1. Adding normalization

- Added normalization
  - lam=0 , acc: 88.54
  - lam=0.0001 , acc: 84.98
  - lam=0.0005 , acc: 91.38
  - lam=0.001 , acc: 94.16
  - lam=0.005 , acc: 85.11
  - lam=0.01 , acc: 73.30
  - lam=0.1 , acc: 9.45
  - lam=1.0 , acc: 8.344
  - lam=10.0 , acc: 10.85
  - lam=100.0 , acc: 9.60

#### 2. ReLU

- Replaced activations with ReLU
  - lan=0m acc: 9.74
  - lam=0.001, acc: 11.82
  - lam=0.01, acc: 9.04
  - lam=0.1, acc: 10.57
  - lam=1.0, acc: 11.96
  - lam=10.0, acc: 9.59
  - lam=100.0, acc: 9.74

#### 3. Leaky ReLU

- Replaced activations with Leaky ReLU
  - lam=0, acc: 10.29
  - lam=0.001, acc: 11.54
  - lam=0.1, acc: 11.54
  - lam=1.0, acc: 11.54
  - lam=100.0, acc: 11.54
- I do not know why these values came out to be the same, but the code seems correct and does not produce the same values for other activation functions. ¯\_(ツ)\_/¯

#### 4. Tanh

- Replaced activations with Tanh
  - lam=0, acc: 83.45
  - lam=0.001, acc: 92.35
  - lam=0.01, acc: 95.97
  - lam=0.1, acc: 91.64
  - lam=1.0, acc: 8.34
  - lam=100.0, acc: 9.18

#### 5. ELU

- Replaced activations with ELU
  - lam=0, acc: 8.62
  - lam=0.001, acc: 8.62
  - lam=0.01, acc: 10.57
  - lam=0.1, acc: 9.87
  - lam=1.0, acc: 10.01

#### 6. Number of iterations

- I found that sigmoid with lam=0.001 and tanh with lam=0.01 performed the best, so I used those for the next tests.
  - sigmoid, 3_000 iterations, lam=0.001, acc: 94.16
  - sigmoid, 10_000 iterations, lam=0.001, acc: 96.24
  - tanh, 3_000 iterations, lam=0.01, acc: 95.97
  - tanh, 10_000 iterations, lam=0.01, acc: 96.11
  - tanh, 30_000 iterations, lam=0.01, acc: 98.19
- It was impractical to continute above 10_000 iterations for sigmoid, so I moved on

#### 7. Changing the number of hidden layers

- tanh, as described above, did the best, and was faster to train, so I went with that
  - lam=0.001, [64, 30, 20, 10], 3_000 iters, acc: 7.37
  - lam=0.001, [64, 30, 30, 10], 3_000 iters, acc: 3.89
  - lam=0.001, [64, 30, 10], 3_000 iters, acc: 91.93
  - lam=0.001, [64, 40, 10], 3_000 iters, acc: 91.93
  - lam=0.001, [64, 20, 10] 3_000 iters, acc: 91.52
  - lam=0.001, [64, 20, 20, 10] 3_000 iters, acc: 7.64
  - lam=0.001, [64, 20, 10] 10_000 iters, acc: 95.83
  - lam=0.01, [64, 20, 10] 30_000 iters, acc: 97.50
- so adding more layers makes it worse, and changing the hidden layer makes it worse. sticking with [64, 30, 10]

#### 8. Changing weight initialization

- Using tanh, lam=0.01, [64, 30, 10], 10_000 iters
  - default, acc: 96.11
  - uniorm, acc: 95.55
  - normal, acc: 96.52

#### 9. Conclusion

- tanh, lam=0.01, [64, 30, 10], with normal weight initialization resulted in the best accuracy
  - At 10_000 iterations, the accuracy was 96.52
  - At 30_000, the accuracy was up to 98
  - Higher iterations may improve performance, but at such a high accuracy already it is unlikely
